In [4]:
import sqlite3
import re
import pandas as pd
import numpy as np

## Clean up subject

In [ ]:
# conn = sqlite3.connect('../hillary-clinton-emails/database.sqlite')
# cur = conn.cursor()
# cur.execute('pragma table_info(Emails)')
# f_1 = 'Id'
# f_2 = 'MetadataTo'
# f_3 = 'MetadataFrom'
# f_4 = 'ExtractedSubject'
# f_5 = 'ExtractedDateSent'
# f_6 = 'ExtractedBodyText'
# f_7 = 'RawText'
# cur.execute('select {c1},{c2},{c3},{c4},{c5},{c6},{c7} from Emails'.\
#             format(c1=f_1,c2=f_2,c3=f_3,c4=f_4,c5=f_5,c6=f_6,c7=f_7))
# cur.fetchall()
# cur.fetchone()
# columns_less = ['id','to','from','sub','date','body','raw']
# columns = [str(tpl[1]) for tpl in columns]
def clean_subject(sub):
    email_type = None
    if re.match('^[Ff][Ww]',sub)!=None:
        email_type='fw'
        sub = re.sub('^[Ff][Ww].*:\s*(.+)',r'\1',sub)
    elif re.match('^[Rr][Ee]:',sub)!=None:
        email_type='re'
        sub = re.sub('^[Rr][Ee]:\s*(.+)',r'\1',sub)
    elif re.match('^[Ff][Vv][Vv]:',sub)!=None:
        email_type='fvv'
        sub = re.sub('^[Ff][Vv][Vv]:\s*(.+)',r'\1',sub)
    return email_type,sub

In [ ]:
conn = sqlite3.connect('../hillary-clinton-emails/database.sqlite')
sql = """select e.Id, p.Name Sender, e.SenderPersonId Sender_Id
, e.MetadataTo Receiver, a.PersonId Receiver_Id, e.ExtractedDateSent, e.ExtractedSubject
from Emails e 
inner join Persons p on e.SenderPersonId=p.Id 
left outer join Aliases a on lower(e.MetadataTo)=a.Alias
"""
emails = pd.read_sql_query(sql,conn)
temp = emails.ExtractedSubject.apply(clean_subject)
emails['ExtractedSubject'] = [tpl[1] for tpl in temp]
emails['email_type'] = email_type = [tpl[0] for tpl in temp]
emails.head(20)

## Clean-up sender receiver

In [ ]:
# Fetch data in pandas dataframe
conn = sqlite3.connect('../hillary-clinton-emails/database.sqlite')
# sql = "select * from Emails"
# emails = pd.read_sql_query(sql,conn)
# columns = ['Id','SenderPersonId','MetadataFrom','MetadataTo','ExtractedDateSent','ExtractedSubject','ExtractedBodyText']
# emails[columns].head(5)
# emails[emails['SenderPersonId']==80][columns].head(5)
# x = emails.groupby('SenderPersonId').size()
# print 'No of mails sent by HC =',x[80]

# Assign receiver Id
sql = """select e.Id, p.Name Sender, 
e.SenderPersonId Sender_Id, e.MetadataTo Receiver, a.PersonId Receiver_Id, e.ExtractedDateSent
from Emails e 
inner join Persons p on e.SenderPersonId=p.Id 
left outer join Aliases a on lower(e.MetadataTo)=a.Alias
"""
emails = pd.read_sql_query(sql,conn)
# emails_noNA = emails.dropna(how='any')
# emails.head(20)
idx_NaN = [i for i,val in enumerate(emails['Receiver_Id']) if np.isnan(val)]


# Create dictionary personId:name
sql = "select * from Persons"
persons = pd.read_sql_query(sql,conn)
persons_dict = {}
for entry in persons.itertuples():
    persons_dict[entry[2]]=entry[1]
    
# Fill up missing Receiver_Id fields
receiver_dict = {}
pattern = re.compile('^(\w+),\sp*(\w+)')
for idx in idx_NaN:
    name = emails['Receiver'][idx]
    pat_obj = pattern.match(name)
    if pat_obj!=None:
        person_name = pat_obj.group(2)+' '+pat_obj.group(1)
        try:
            person_id = persons_dict[person_name]
            emails['Receiver_Id'][idx]=person_id
        except:
            pass
temp = emails.shape[0]
emails = emails.dropna(how='any')
print('Discarded emails: ',temp-emails.shape[0])

emails.head(20)

## Clean up date

In [6]:
# txt = 'Thursday, September 30, 2010 9:04 PM'
# txt = 'Thu Sep 17 06:03:43 2009'
def clean_date_aux(ts):
    pat = re.compile('\'?(?P<weekday>\w+),?\s*(?P<month>\w+) (?P<date>\d+),?\s*(?P<year>\d+) (?P<time>.+) (?P<ampm>[A|P]M)')
    pat1 = re.compile('(?P<weekday>.+) (?P<month>.+) (?P<date>.+) (?P<time>.+) (?P<year>.+)')
    if ts=='':
        return np.NAN
    re_obj = None
    try:
        if pat.match(ts)!=None:
            re_obj = pat.match(ts)
        elif pat1.match(ts)!=None:
            re_obj = pat1.match(ts)
        else:
            raise
        # Process date
        month = re_obj.group('month')
        day = re_obj.group('date')
        year = re_obj.group('year')
        time = re_obj.group('time').replace('.',':')
        # Check for numeric value of day and year
        try:
            month = month[:3]
            day = int(day)
            year = int(year)
        except:
            raise
        # Check time string
        if ':' not in time and len(time)>2:
            time = time[:-2]+':'+time[-2:]
        try:
            time = time + ' ' + re_obj.group('ampm')
        except:
            pass
        ts_str = "{} {} {} {}".format(month,day,year,time)
        try:
            pd_ts = pd.to_datetime(ts_str)
            return pd_ts
        except:
            raise
    except:
        return np.NAN

In [7]:
def clean_date(raw_date):
    return clean_date_aux(raw_date)
emails['Date']=emails.ExtractedDateSent.apply(clean_date)
emails.head(10)
# emails = emails.dropna(how='any')
emails.index = emails.Date
emails.sort_index(inplace=True)
mindate,maxdate = emails.index.min(),emails.index.max()
"Date Range from {} to {}".format(mindate.date(),maxdate.date())
emails[:'2011-01'].Id.count()

6306

## Process email body: Topic modelling

In [8]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [9]:
class HillaryEmails(object):
    def __init__(self,doc_vectors):
        self.doc_vectors = doc_vectors
    def __iter__(self):
        for vector in self.doc_vectors:
            yield vector

In [44]:
def tokenize(text):
    text = text.replace("\n"," ")
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]
def do_BoW(tokens):
    return vocab.doc2bow(tokens)
def get_maxwords(vector):
    minimum = int(len(vector)*0.2)
    minimum = min(10,minimum)
    global vocab
    temp = sorted(vector,key=lambda x:x[1],reverse=True)
    topic = [vocab[it[0]] for it in temp[:minimum]]
    return topic
conn = sqlite3.connect('../hillary-clinton-emails/database.sqlite')
sql = """select e.Id, p.Name Sender, 
e.SenderPersonId Sender_Id, e.ExtractedDateSent, e.ExtractedBodyText
from Emails e 
inner join Persons p on e.SenderPersonId=p.Id
"""
emails = pd.read_sql_query(sql,conn)
#print(emails)
emails['ExtractedDateSent']=emails.ExtractedDateSent.apply(clean_date)
#print(emails['ExtractedDateSent'])
# Step 1: Mails from Hillary
emails = emails[emails['Sender_Id']==80]
#print(emails)
# Step 2: Tokenize email body
email_tokens = emails.ExtractedBodyText.apply(tokenize)
#print(email_tokens)
doc_stream = [tokens for tokens in email_tokens]
#print(doc_stream)
print(len(doc_stream))
# Step 3: Build Vocabulary
%time vocab = gensim.corpora.Dictionary(doc_stream)
#print(vocab)
    # print vocab.items()
    # vocab.filter_extremes(no_below=20,no_above=0.8)
    # vocab.filter_extremes(no_above=0.5)
# Step 4: Run BoW on docs
vectors_doc = email_tokens.apply(do_BoW)
#print(vectors_doc)
maxwords = vectors_doc.apply(get_maxwords)
#print("max\n",maxwords)
#Step 5: save doc vectors in mm file
hillary_emails_corpus = HillaryEmails(vectors_doc)
#print(hillary_emails_corpus)
%time gensim.corpora.MmCorpus.serialize('emails_bow.mm',hillary_emails_corpus)

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : built Dictionary(4249 unique tokens: ['intervention', 'thomases', 'denigration', 'board', 'ibrahim']...) from 1993 documents (total 17317 corpus positions)
INFO : storing corpus in Matrix Market format to emails_bow.mm
INFO : saving sparse matrix to emails_bow.mm
INFO : PROGRESS: saving document #0
INFO : PROGRESS: saving document #1000
INFO : saved 1993x4249 matrix, density=0.187% (15844/8468257)
INFO : saving MmCorpus index to emails_bow.mm.index


1993
Wall time: 31 ms
Wall time: 44 ms


In [31]:
hillary_mm_corpus = gensim.corpora.MmCorpus('emails_bow.mm')
print(hillary_mm_corpus)
clipped_corpus = gensim.utils.ClippedCorpus(hillary_emails_corpus,1600)
%time lda_model = gensim.models.LdaModel(clipped_corpus, num_topics=20, id2word=vocab, passes=4)

INFO : loaded corpus index from emails_bow.mm.index
INFO : initializing corpus reader from emails_bow.mm
INFO : accepted corpus with 1993 documents, 4249 features, 15844 non-zero entries
INFO : using symmetric alpha at 0.05
INFO : using symmetric eta at 0.05
INFO : using serial LDA version on this node


MmCorpus(1993 documents, 4249 features, 15844 non-zero entries)


INFO : running online LDA training, 20 topics, 4 passes over the supplied corpus of 1600 documents, updating model once every 1600 documents, evaluating perplexity every 1600 documents, iterating 50x with a convergence threshold of 0.001000
INFO : -20.091 per-word bound, 1116939.7 perplexity estimate based on a held-out corpus of 1600 documents with 14746 words
INFO : PROGRESS: pass 0, at document #1600/1600
INFO : topic #19 (0.050): 0.008*ll + 0.007*talk + 0.006*email + 0.005*supposed + 0.005*mtg + 0.005*yes + 0.005*state + 0.005*like + 0.004*good + 0.004*want
INFO : topic #11 (0.050): 0.015*print + 0.013*pis + 0.008*asked + 0.008*tomorrow + 0.007*com + 0.007*clintonemail + 0.007*good + 0.006*pls + 0.006*hrod + 0.005*thanks
INFO : topic #7 (0.050): 0.017*know + 0.016*ok + 0.014*print + 0.013*state + 0.012*talk + 0.012*pis + 0.012*pls + 0.012*hrod + 0.012*com + 0.012*pm
INFO : topic #15 (0.050): 0.032*state + 0.026*hrod + 0.026*gov + 0.026*com + 0.025*clintonemail + 0.018*pm + 0.012*fw

Wall time: 8.4 s


In [32]:
lda_model.print_topics(-1)

INFO : topic #0 (0.050): 0.019*ok + 0.015*know + 0.013*want + 0.011*calling + 0.009*thank + 0.009*thx + 0.009*send + 0.008*hope + 0.008*need + 0.007*btw
INFO : topic #1 (0.050): 0.019*fyi + 0.018*today + 0.015*want + 0.014*gov + 0.013*good + 0.011*state + 0.010*com + 0.010*tomorrow + 0.010*clintonemail + 0.008*come
INFO : topic #2 (0.050): 0.017*pm + 0.016*state + 0.011*like + 0.009*office + 0.009*department + 0.007*ok + 0.007*time + 0.007*think + 0.006*house + 0.006*romney
INFO : topic #3 (0.050): 0.027*pls + 0.016*set + 0.015*today + 0.010*diplomacy + 0.009*thought + 0.008*list + 0.008*heard + 0.008*team + 0.007*add + 0.007*told
INFO : topic #4 (0.050): 0.020*tomorrow + 0.020*pls + 0.015*thx + 0.014*talk + 0.013*time + 0.011*let + 0.011*like + 0.011*want + 0.010*schedule + 0.009*monday
INFO : topic #5 (0.050): 0.017*yes + 0.014*thx + 0.013*know + 0.013*let + 0.009*state + 0.007*said + 0.007*time + 0.007*want + 0.007*gov + 0.006*tomorrow
INFO : topic #6 (0.050): 0.017*pis + 0.012*pls 

[(0,
  '0.019*ok + 0.015*know + 0.013*want + 0.011*calling + 0.009*thank + 0.009*thx + 0.009*send + 0.008*hope + 0.008*need + 0.007*btw'),
 (1,
  '0.019*fyi + 0.018*today + 0.015*want + 0.014*gov + 0.013*good + 0.011*state + 0.010*com + 0.010*tomorrow + 0.010*clintonemail + 0.008*come'),
 (2,
  '0.017*pm + 0.016*state + 0.011*like + 0.009*office + 0.009*department + 0.007*ok + 0.007*time + 0.007*think + 0.006*house + 0.006*romney'),
 (3,
  '0.027*pls + 0.016*set + 0.015*today + 0.010*diplomacy + 0.009*thought + 0.008*list + 0.008*heard + 0.008*team + 0.007*add + 0.007*told'),
 (4,
  '0.020*tomorrow + 0.020*pls + 0.015*thx + 0.014*talk + 0.013*time + 0.011*let + 0.011*like + 0.011*want + 0.010*schedule + 0.009*monday'),
 (5,
  '0.017*yes + 0.014*thx + 0.013*know + 0.013*let + 0.009*state + 0.007*said + 0.007*time + 0.007*want + 0.007*gov + 0.006*tomorrow'),
 (6,
  '0.017*pis + 0.012*pls + 0.011*thx + 0.010*state + 0.009*hrod + 0.009*tell + 0.009*fax + 0.008*year + 0.008*com + 0.008*emai

In [33]:
%time tfidf_model = gensim.models.TfidfModel(hillary_mm_corpus, id2word=vocab)
%time lsi_model = gensim.models.LsiModel(tfidf_model[hillary_mm_corpus], id2word=vocab, num_topics=200)

INFO : collecting document frequencies
INFO : PROGRESS: processing document #0
INFO : calculating IDF weights for 1993 documents and 4248 features (15844 matrix non-zeros)
INFO : using serial LSI version on this node
INFO : updating model with new documents
INFO : preparing a new chunk of documents
INFO : using 100 extra samples and 2 power iterations
INFO : 1st phase: constructing (4249, 300) action matrix
INFO : orthonormalizing (4249, 300) action matrix


Wall time: 48 ms


INFO : 2nd phase: running dense svd on (300, 1993) matrix
INFO : computing the final decomposition
INFO : keeping 200 factors (discarding 12.506% of energy spectrum)
INFO : processed documents up to #1993
INFO : topic #0(9.460): 0.995*"ok" + 0.074*"thx" + 0.033*"pls" + 0.024*"print" + 0.017*"talk" + 0.010*"dc" + 0.009*"pis" + 0.008*"let" + 0.008*"clintonemail" + 0.008*"tomorrow"
INFO : topic #1(8.125): 0.787*"print" + 0.530*"pls" + 0.268*"pis" + 0.049*"gov" + 0.047*"state" + -0.045*"ok" + 0.044*"hrod" + 0.044*"clintonemail" + 0.043*"copies" + 0.043*"com"
INFO : topic #2(5.791): 0.869*"thx" + 0.144*"state" + 0.143*"gov" + 0.139*"hrod" + 0.139*"clintonemail" + 0.138*"com" + 0.109*"pm" + 0.098*"yes" + -0.096*"print" + 0.085*"talk"
INFO : topic #3(5.447): -0.429*"thx" + 0.337*"gov" + 0.335*"state" + 0.309*"hrod" + 0.308*"clintonemail" + 0.307*"com" + 0.235*"pm" + 0.174*"fw" + 0.136*"jilotylc" + -0.113*"pls"
INFO : topic #4(4.831): -0.807*"pis" + 0.551*"pls" + -0.120*"print" + 0.045*"state"

Wall time: 383 ms


## Running R from python

In [ ]:
'''import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
base = importr('base')
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)
'''

In [ ]:
'''packnames = ['ggplot2']
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install)>0:
    utils.install_packages(StrVector(names_to_install))
    '''

## Sentiment Analysis on countries

In [36]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

In [37]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
n_instances = 100
subj_docs = [(sent,'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent,'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
# subj_docs[10],obj_docs[1]
train_docs = subj_docs[:80]+obj_docs[:80]
test_docs = subj_docs[80:100]+obj_docs[80:100]
senti_analyser = SentimentAnalyzer()

In [38]:
sent = "He is a murderer"
sentiAnalyser = SentimentIntensityAnalyzer()
sentiAnalyser.polarity_scores(sent)

{'compound': -0.6808, 'neg': 0.697, 'neu': 0.303, 'pos': 0.0}

## Find sentiment of emails

In [39]:
def getSentiment(text):
    pos = 0
    neg = 0
    neu = 0
    if text=='':
        return 0,pos,neg
    global sentiAnalyser
    text = text.replace('\n',' ')
    sentences = sent_tokenize(text)
    sentences = [sent for sent in sentences if len(sent.split())>2]
    for sent in sentences:
        senti = sentiAnalyser.polarity_scores(sent)
        if senti['neu']>=0.9:
            neu += 1
        else:
            pos += senti['pos']
            neg += senti['neg']
    count = len(sentences)-neu
    if count == 0:
        return len(sentences),pos,neg
    return len(sentences),pos/count,neg/count
conn = sqlite3.connect('../hillary-clinton-emails/database.sqlite')
sql = """select e.Id, p.Name Sender, 
e.SenderPersonId Sender_Id, e.ExtractedDateSent, e.ExtractedBodyText
from Emails e 
inner join Persons p on e.SenderPersonId=p.Id
"""
emails = pd.read_sql_query(sql,conn)
emails['ExtractedDateSent']=emails.ExtractedDateSent.apply(clean_date)
# Step 1: Mails from Hillary
emails = emails[emails['Sender_Id']==80]
# emails = emails.head(5)
# emails.ExtractedBodyText.apply(getSentiment)
print('Total Email count by Hillary: ',emails.Id.count())
sentiAnalyser = SentimentIntensityAnalyzer()
%time sentiments = emails.ExtractedBodyText.apply(getSentiment)
senti_map = zip(emails.Id.values,sentiments.values)
senti_map = sorted(senti_map,key = lambda item:item[1][2],reverse=True)
print(senti_map[:5])

Total Email count by Hillary:  1993
Wall time: 743 ms
[(552, (7, 0.0, 0.772)), (2240, (1, 0.0, 0.7)), (585, (3, 0.0, 0.531)), (1022, (2, 0.0, 0.508)), (2308, (1, 0.0, 0.483))]


## Find occurance of country name in mails

In [40]:
import pycountry

ImportError: No module named 'pycountry'

In [ ]:
def get_count(text):
    global country_counter
    global countries_new_name
    for indx,country in enumerate(pycountry.countries):
        name = country.name
        if name in countries_new_name.keys():
            name = countries_new_name[name]
        name = '\\b'+name # Ensuring substring start with country name. eg.Indian, Syrian, India
        all_matches = re.finditer(name,text,re.IGNORECASE)
        n_occurrences = len(list(all_matches))
        country_counter[indx] += n_occurrences
country_counter = [0 for country in pycountry.countries]
country_names = [country.name for country in pycountry.countries]
countries_new_name = {'Syrian Arab Republic':'Syria','Russian Federation':'Russia','Iran, Islamic Republic of':\
                      'Iran','Korea, Democratic People\'s Republic of':'Korea','Palestine, State of':'Palestine',\
                      'Venezuela, Bolivarian Republic of':'Venezuela'}
emails.ExtractedBodyText.apply(get_count)
# let's study count for some countries
country2ind = {}
for indx,country in enumerate(pycountry.countries):
    country2ind[country.name]=indx
country_name_count = zip(country_names,country_counter)
country_name_count = sorted(country_name_count,key = lambda(item):item[1],reverse=True)

In [ ]:
%matplotlib inline
df = pd.DataFrame(np.array(country_counter),country_names)
df = df[df[0]>1]
ax = df.plot(kind='bar',legend=False,title='Occurance of Country names')
ax.set_ylabel('Count')

## Correlation between country vs average sentiment

In [ ]:
def get_occurence_ct(text,country):
    name = country.name
    counter = 0
    global countries_new_name
    if name in countries_new_name.keys():
        name = countries_new_name[name]
    name = '\\b'+name # Ensuring substring start with country name. eg.Indian, Syrian, India
    all_matches = re.finditer(name,text,re.IGNORECASE)
    n_occurrences = len(list(all_matches))
    counter += n_occurrences
    abbr_to_ignore = ["RE", "FM", "TV", "AL", "AQ", "LA", "BEN"]
    if country.alpha2 not in abbr_to_ignore:
        name = '\\b'+country.alpha2+'\\b'
        all_matches = re.finditer(name,text)
        n_occurrences = len(list(all_matches))
        counter += n_occurrences
    if country.alpha3 not in abbr_to_ignore:
        name = '\\b'+country.alpha3+'\\b'
        all_matches = re.finditer(name,text)
        n_occurrences = len(list(all_matches))
        counter += n_occurrences
    return counter
def get_country_sentiment(country):
    global sentiments
    global emails
    res_pos,res_neg = 0,0
    counts = emails.ExtractedBodyText.apply(get_occurence_ct,args=(country,))
    sentiments_pos = [tpl[1] for tpl in sentiments.values]
    sentiments_neg = [tpl[2] for tpl in sentiments.values]
    if sum(counts.values)>0:
        res_pos = sum([ct*pos for ct,pos in zip(counts.values,sentiments_pos)])/sum(counts.values)
        res_neg = sum([ct*neg for ct,neg in zip(counts.values,sentiments_neg)])/sum(counts.values)
    return res_pos,res_neg
countries = pd.Series([country for country in pycountry.countries])
senti = countries.apply(get_country_sentiment)

In [ ]:
senti_pos = np.array([i[0] for i in senti])
df = pd.DataFrame(senti_pos,country_names)
df = df[df[0]>0.1]
ax = df.plot(kind='bar',legend=False,title='Hillary\'s Sentiment about countries')
ax.set_ylabel('postivity')

In [ ]:
senti_neg = np.array([i[1] for i in senti])
df = pd.DataFrame(senti_neg,country_names)
df = df[df[0]>0]
ax = df.plot(kind='bar',legend=False,title='Hillary\'s Sentiment about countries')
ax.set_ylabel('negativity')